<a href="https://colab.research.google.com/github/HayatoYagi/Elliptic-Curves/blob/main/elliptic_curve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def divisors(n):
  res = []
  i = 1
  while(i * i < n):
    if n % i == 0:
      res.append(i)
      res.append(n // i)
    i += 1
  if i * i == n:
    res.append(i)
  return res

class EllipticCurve:
  def __init__(self, a, b, c):
    self._a = a
    self._b = b
    self._c = c
  
  def discriminant(self):
    return (-4 * self._a * self._a * self._a * self._c
            + self._a * self._a * self._b * self._b
            + 18 * self._a * self._b * self._c
            - 4 * self._b * self._b * self._b
            - 27 * self._c * self._c)
  
  def y_candidates(self):
    res = [0]
    D = self.discriminant()
    assert(D < 0)
    i = 1
    while i * i <= -D:
      if(-D % (i * i) == 0):
        res.append(i)
      i += 1
    return res

  def find_x(self, y):
    res = []
    a = self._a
    b = self._b
    c = self._c - y * y
    for d in divisors(abs(c)):
      if(d * d * d + a * d * d + b * d + c == 0):
        res.append(d)
      if(- d * d * d + a * d * d - b * d + c == 0):
        res.append(-d)
    return res

  def candidates(self):
    res = []
    for y in self.y_candidates():
      for x in self.find_x(y):
        res.append((x,y))
        if(y != 0):
          res.append((x,-y))
    return res

In [3]:
e1 = EllipticCurve(0,-43,166)
print(e1.candidates())

[(3, 8), (3, -8), (-5, 16), (-5, -16), (11, 32), (11, -32)]


In [4]:
e2 = EllipticCurve(0,0,1)
print(e2.candidates())

[(-1, 0), (2, 3), (2, -3)]
